# 1.1 Analisis Exploratorio de datos

Cargamos los datos inicialmente

In [30]:
import pandas as pd

data_path = 'data/home_data.csv'
df = pd.read_csv(data_path)

# vemos el esquema del Dataframe para darnos una idea inicial de las columnas y tipos de datos
print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  int64  
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

Podemos ver que tenemos un dataframe con 21613 registros, 21 columnas, y que no hay valores nulos en cada columna inicialmente

Para el EDA, Personalmente me gusta usar una libreria llamada ydata_profiling, allí puedo evaluar de manera integra la estructura de los datos en un .html que se genera

In [13]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="data_precio_viviendas ")
profile.to_file("data_precio_viviendas.html")


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 25.36it/s]


Podemos ver el resumen estadistico del df, el tipo de variables y la cantidad de memoria que ocupa el dataset

![overview](img/overview.png)

además podemos ver un resumen por cada columna para identificar aspectos importantes:

columna id

![id](img/resumenId.png)

columna Price

![price](img/resumenPrice.png)

columna bathrooms

![bathroom](img/resumenBathrooms.png)

columna Bedrooms

![bedrooms](img/resumenBedrooms.png)

columna floors

![floors](img/resumenFloors.png)

columna sqft_above

![sqft_above](img/resumensqft_above.png)

columna sqft_basement

![sqft_basement](img/resumensqft_basement.png)

columna sqft_living

![sqft_living](img/resumensqft_living.png)

columna sqtf_lot

![sqft_lot](img/resumensqft_lot.png)

![correlaciones](img/resumenCorrelaciones.png)



In [31]:
# exploremos las correlaciones solo del precio con las demás variables numéricas
numeric_df = df.select_dtypes(include=['number'])

# Calcular correlaciones con 'price'
correlations = numeric_df.corr()['price'].sort_values(ascending=False)

print("\n🔗 Top 10 Correlaciones con el Precio:")
print(correlations.head(11))


🔗 Top 10 Correlaciones con el Precio:
price            1.000000
sqft_living      0.702035
grade            0.667434
sqft_above       0.605567
sqft_living15    0.585379
bathrooms        0.525138
view             0.397293
sqft_basement    0.323816
bedrooms         0.308350
lat              0.307003
waterfront       0.266369
Name: price, dtype: float64



## conclusiones 1.1 EDA 
* existen algunos registros duplicados en el id que se tienen que depurar en la parte subsiguiente de preprocesamiento de datos
* es necesario cambiar el tipo de dato de la columna bathrooms y floors ya que al ser baños y pisos respectivamente, no pueden existir decimales, se debe redondear al entero más cercano y convertir a tipo int
* depurar los outliers de la columna bedrooms ya que hay algunos registros que presentan hasta 33 dormitorios, lo cual puede sesgar los modelos, es necesario eliminarlos
* depurar los outliers de la columna sqft_lot ya que existen valores muy altos que no permiten conocer bien la distribucion de los datos y generan sesgos
* adicionalmente:
   - Relación no lineal entre algunas variables y precio
   - Posible multicolinealidad entre variables de área
   - sqft_living: Correlación más fuerte con precio (>0.7)
   - grade: segunda relacion más fuerte, aunque no se explica mucho que es esta variable, es importante.


# 1.2 preprocesamiento datos

Miremos cuantos duplicados hay en la columna "id", deberían tener valores unicos

In [32]:
duplicados = df[df.duplicated(subset='id', keep=False)]
print(f"Número de registros duplicados por 'id': {duplicados['id'].nunique()}")

Número de registros duplicados por 'id': 176


Aqui nos dimos cuenta que hay varios registros duplicados, miremos inicialmente cual puede ser el motivo

In [33]:
# genero un array con los ids duplicados
duplicados_ids = df['id'][df['id'].duplicated()].unique()

# quiero ver un ejemplo de los duplicados
id_ejemplo = duplicados_ids[2]
df_id = df[df['id'] == id_ejemplo]
df_id

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
324,7520000520,20140905T000000,232000,2,1.0,1240,12092,1.0,0,0,...,6,960,280,1922,1984,98146,47.4957,-122.352,1820,7460
325,7520000520,20150311T000000,240500,2,1.0,1240,12092,1.0,0,0,...,6,960,280,1922,1984,98146,47.4957,-122.352,1820,7460


a priori parece que las columnas que difieren son la fecha y el precio, lo cual sugiere que hay registros en los que se actualizó el precio del inmueble en una fecha posterior, validemos esta hipótesis para todos los registros

In [17]:
diferencias = {}

for _id in duplicados_ids:
    filas = df[df['id'] == _id]
    if len(filas) > 1:
        diff = filas.loc[filas.index[0]] != filas.loc[filas.index[1]]
        cols_diff = diff[diff].index.tolist()
        diferencias[_id] = cols_diff

# Mostrar IDs y columnas distintas
for k, v in list(diferencias.items())[:10]:  # solo los primeros 10 para no saturar
    print(f"ID {k} difiere en: {v}")

ID 6021501535 difiere en: ['date', 'price']
ID 4139480200 difiere en: ['date', 'price']
ID 7520000520 difiere en: ['date', 'price']
ID 3969300030 difiere en: ['date', 'price']
ID 2231500030 difiere en: ['date', 'price']
ID 8820903380 difiere en: ['date', 'price']
ID 726049190 difiere en: ['date', 'price']
ID 8682262400 difiere en: ['date', 'price']
ID 9834200885 difiere en: ['date', 'price']
ID 8062900070 difiere en: ['date', 'price']


Efectivamente, las columnas que hacen que estén duplicados son por fecha y precio, como queremos generar un modelo de precios, eliminaremos los registros viejos y dejaremos los mas recientes

ordenamos entonces el dataframe para dejar los registros más actualizados.



In [37]:
# Ordenar por id y fecha (de más reciente a más antigua)
df_sorted = df.sort_values(['id', 'date'], ascending=[True, False])

# Eliminar duplicados, manteniendo SOLO el registro más reciente
df_clean = df_sorted.drop_duplicates(subset='id', keep='first')

# eliminiamos también el que tuvo 33 bedrooms porque es un error obvio
df_clean = df_clean[df_clean['bedrooms'] < 33]

#convertimos la columna date a tipo fecha
df_clean['date'] = df_clean['date'].astype(str)

# convierto a tipo int la columna de bathrooms y floors 
df_clean['bathrooms'] = df_clean['bathrooms'].astype(int)
df_clean['floors'] = df_clean['floors'].astype(int)

In [19]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df_clean, title="precio vivendas depurado ")
profile.to_file("data_precio_viviendas_depurado.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 29.77it/s]


A partir de acá guardo en un archivo .parquet el dataset ya depurado

In [43]:
df_clean.to_parquet('data/home_data_clean.parquet', index=False,engine='fastparquet')
#df_clean.info()

## Conclusiones 1.2 Preprocesamiento de datos:
* se eliimnaron id's duplicados
* se eliminaron los outliers que no tenian sentido como los 33 bedrooms
* variables como floors o bathrooms se transformaron a enteros debido a la naturaleza del dato

# 1.3 Modelo para ajustar el precio de las viviendas

In [ ]:
# haremos tratamientos adicionales al set de datos crudos para el modelado, no se hicieron antes para poder analizar los datos en su estado original.
# 2.2 Tratamiento de outliers en precio

# Crear copia para preprocesamiento
df_processed = df_clean.copy()

Q1 = df_processed['price'].quantile(0.25)
Q3 = df_processed['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

outliers_before = len(df_processed)
df_processed = df_processed[(df_processed['price'] >= lower_bound) & 
                            (df_processed['price'] <= upper_bound)]
outliers_removed = outliers_before - len(df_processed)
print(f"   - Outliers removidos: {outliers_removed} ({outliers_removed/outliers_before*100:.2f}%)")

   - Outliers removidos: 422 (1.97%)


In [47]:
# Edad de la vivienda
if 'yr_built' in df_processed.columns:
    df_processed['house_age'] = 2025 - df_processed['yr_built']
    print("   - Creada: house_age (edad de la vivienda)")

# Fue renovada
if 'yr_renovated' in df_processed.columns:
    df_processed['is_renovated'] = (df_processed['yr_renovated'] > 0).astype(int)
    print("   - Creada: is_renovated (indicador de renovación)")

# Precio por sqft
if 'sqft_living' in df_processed.columns:
    df_processed['price_per_sqft'] = df_processed['price'] / df_processed['sqft_living']
    print("   - Creada: price_per_sqft (precio por pie cuadrado)")

# Ratio baños/habitaciones
if 'bathrooms' in df_processed.columns and 'bedrooms' in df_processed.columns:
    df_processed['bath_bed_ratio'] = df_processed['bathrooms'] / (df_processed['bedrooms'] + 1)
    print("   - Creada: bath_bed_ratio (ratio baños/habitaciones)")

# Eliminar columnas no relevantes o de identificación
columns_to_drop = ['id', 'date', 'yr_renovated', 'zipcode']
existing_drops = [col for col in columns_to_drop if col in df_processed.columns]
df_processed = df_processed.drop(columns=existing_drops)
print(f"   - Columnas eliminadas: {existing_drops}")

   - Creada: house_age (edad de la vivienda)
   - Creada: is_renovated (indicador de renovación)
   - Creada: price_per_sqft (precio por pie cuadrado)
   - Creada: bath_bed_ratio (ratio baños/habitaciones)
   - Columnas eliminadas: ['id', 'date', 'yr_renovated', 'zipcode']


In [49]:
df_processed.head(5)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,lat,long,sqft_living15,sqft_lot15,house_age,is_renovated,price_per_sqft,bath_bed_ratio
2497,300000,6,3,2400,9373,2,0,0,3,7,...,0,1991,47.3262,-122.214,2060,7316,34,0,125.000000,0.428571
6735,647500,4,1,2060,26036,1,0,0,4,8,...,900,1947,47.4444,-122.351,2590,21891,78,0,314.320388,0.200000
8411,400000,3,1,1460,43000,1,0,0,3,7,...,0,1952,47.4434,-122.347,2250,20023,73,0,273.972603,0.250000
8809,235000,3,1,1430,7599,1,0,0,4,6,...,420,1930,47.4783,-122.265,1290,10320,95,0,164.335664,0.250000
3557,402500,4,2,1650,3504,1,0,0,3,7,...,890,1951,47.5803,-122.294,1480,3504,74,1,243.939394,0.400000


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# 2.5 División de datos
print("\n🔧 Paso 5: División Train/Test")
X = df_processed.drop('price', axis=1) # Caracteristicas
y = df_processed['price'] # Objetivo a predecir

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"   - Train set: {X_train.shape[0]} muestras")
print(f"   - Test set: {X_test.shape[0]} muestras")


🔧 Paso 5: División Train/Test
   - Train set: 16810 muestras
   - Test set: 4203 muestras


In [57]:
# 2.6 Escalado de características
print("\n🔧 Paso 6: Escalado de Características")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("   ✅ Características escaladas con StandardScaler")

print("\n" + "="*80)
print("📝 RESUMEN DEL PREPROCESAMIENTO")
print("="*80)
print(f"""
- Datos originales: {outliers_before:,} filas
- Datos procesados: {len(df_processed):,} filas
- Características finales: {X_train.shape[1]}
- Nuevas features creadas: 4
- Estrategia de escalado: StandardScaler
""")


🔧 Paso 6: Escalado de Características
   ✅ Características escaladas con StandardScaler

📝 RESUMEN DEL PREPROCESAMIENTO

- Datos originales: 21,435 filas
- Datos procesados: 21,013 filas
- Características finales: 20
- Nuevas features creadas: 4
- Estrategia de escalado: StandardScaler



usaremos varios modelos:
* Regresión lineal inicialmente
* Regresión Ridge
* Regresión Lasso
* Random forest
* Gradient Boosting

In [66]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
print("\n" + "="*80)
print("3. MODELADO Y SELECCIÓN DEL MEJOR MODELO")
print("="*80)

# Diccionario para almacenar resultados
results = {}

# 3.1 Regresión Lineal
print("\n🤖 Modelo 1: Regresión Lineal")
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

results['Linear Regression'] = {
    'model': lr,
    'y_pred': y_pred_lr,
    'rmse': np.sqrt(mean_squared_error(y_test, y_pred_lr)),
    'mae': mean_absolute_error(y_test, y_pred_lr),
    'r2': r2_score(y_test, y_pred_lr),
    'cv_score': cross_val_score(lr, X_train_scaled, y_train, cv=5, 
                                 scoring='neg_mean_squared_error').mean()
}

print(f"   - RMSE: ${results['Linear Regression']['rmse']:,.2f}")
print(f"   - MAE: ${results['Linear Regression']['mae']:,.2f}")
print(f"   - R²: {results['Linear Regression']['r2']:.4f}")


# 3.2 Ridge Regression
print("\n🤖 Modelo 2: Ridge Regression")
ridge = Ridge(alpha=10)
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_scaled)

results['Ridge'] = {
    'model': ridge,
    'y_pred': y_pred_ridge,
    'rmse': np.sqrt(mean_squared_error(y_test, y_pred_ridge)),
    'mae': mean_absolute_error(y_test, y_pred_ridge),
    'r2': r2_score(y_test, y_pred_ridge),
    'cv_score': cross_val_score(ridge, X_train_scaled, y_train, cv=5,
                                 scoring='neg_mean_squared_error').mean()
}

print(f"   - RMSE: ${results['Ridge']['rmse']:,.2f}")
print(f"   - MAE: ${results['Ridge']['mae']:,.2f}")
print(f"   - R²: {results['Ridge']['r2']:.4f}")

# 3.3 Lasso Regression
print("\n🤖 Modelo 3: Lasso Regression")
lasso = Lasso(alpha=10)
lasso.fit(X_train_scaled, y_train)
y_pred_lasso = lasso.predict(X_test_scaled)

results['Lasso'] = {
    'model': lasso,
    'y_pred': y_pred_lasso,
    'rmse': np.sqrt(mean_squared_error(y_test, y_pred_lasso)),
    'mae': mean_absolute_error(y_test, y_pred_lasso),
    'r2': r2_score(y_test, y_pred_lasso),
    'cv_score': cross_val_score(lasso, X_train_scaled, y_train, cv=5,
                                 scoring='neg_mean_squared_error').mean()
}

print(f"   - RMSE: ${results['Lasso']['rmse']:,.2f}")
print(f"   - MAE: ${results['Lasso']['mae']:,.2f}")
print(f"   - R²: {results['Lasso']['r2']:.4f}")

# 3.4 Random Forest
print("\n🤖 Modelo 4: Random Forest Regressor")
rf = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

results['Random Forest'] = {
    'model': rf,
    'y_pred': y_pred_rf,
    'rmse': np.sqrt(mean_squared_error(y_test, y_pred_rf)),
    'mae': mean_absolute_error(y_test, y_pred_rf),
    'r2': r2_score(y_test, y_pred_rf),
    'cv_score': cross_val_score(rf, X_train, y_train, cv=5,
                                 scoring='neg_mean_squared_error').mean()
}

print(f"   - RMSE: ${results['Random Forest']['rmse']:,.2f}")
print(f"   - MAE: ${results['Random Forest']['mae']:,.2f}")
print(f"   - R²: {results['Random Forest']['r2']:.4f}")

# 3.5 Gradient Boosting
print("\n🤖 Modelo 5: Gradient Boosting Regressor")
gb = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

results['Gradient Boosting'] = {
    'model': gb,
    'y_pred': y_pred_gb,
    'rmse': np.sqrt(mean_squared_error(y_test, y_pred_gb)),
    'mae': mean_absolute_error(y_test, y_pred_gb),
    'r2': r2_score(y_test, y_pred_gb),
    'cv_score': cross_val_score(gb, X_train, y_train, cv=5,
                                 scoring='neg_mean_squared_error').mean()
}

print(f"   - RMSE: ${results['Gradient Boosting']['rmse']:,.2f}")
print(f"   - MAE: ${results['Gradient Boosting']['mae']:,.2f}")
print(f"   - R²: {results['Gradient Boosting']['r2']:.4f}")

# Comparación de modelos
print("\n" + "="*80)
print("📊 COMPARACIÓN DE MODELOS")
print("="*80)

comparison_df = pd.DataFrame({
    'Modelo': list(results.keys()),
    'RMSE': [results[m]['rmse'] for m in results.keys()],
    'MAE': [results[m]['mae'] for m in results.keys()],
    'R²': [results[m]['r2'] for m in results.keys()],
    'CV Score (MSE)': [-results[m]['cv_score'] for m in results.keys()]
})

print(comparison_df.to_string(index=False))

# Seleccionar mejor modelo
best_model_name = min(results.keys(), key=lambda x: results[x]['rmse'])
best_model = results[best_model_name]['model']

print(f"\n🏆 MEJOR MODELO: {best_model_name}")
print(f"   - RMSE: ${results[best_model_name]['rmse']:,.2f}")
print(f"   - R²: {results[best_model_name]['r2']:.4f}")

# ============================================================================
# 4. ESTRATEGIA DE INVERSIÓN
# ============================================================================

print("\n" + "="*80)
print("4. IDENTIFICACIÓN DE VIVIENDAS ATRACTIVAS PARA INVERSIÓN")
print("="*80)

print("""
📈 ESTRATEGIA DE INVERSIÓN:

Una vivienda es atractiva para inversión cuando:
1. El precio de mercado está SUB-VALUADO respecto a la predicción del modelo
2. Tiene características deseables (buena ubicación, condición, tamaño)
3. El diferencial entre precio predicho y real es significativo (>10%)

Fórmula: Potencial de Inversión = (Precio Predicho - Precio Real) / Precio Real * 100
""")

# Calcular predicciones para todo el conjunto de prueba
if best_model_name in ['Linear Regression', 'Ridge', 'Lasso']:
    y_pred_best = best_model.predict(X_test_scaled)
else:
    y_pred_best = best_model.predict(X_test)

# Crear DataFrame de inversión
investment_df = X_test.copy()
investment_df['precio_real'] = y_test.values
investment_df['precio_predicho'] = y_pred_best
investment_df['diferencia'] = investment_df['precio_predicho'] - investment_df['precio_real']
investment_df['potencial_pct'] = (investment_df['diferencia'] / investment_df['precio_real']) * 100

# Filtrar oportunidades (subvaluadas > 10%)
opportunities = investment_df[investment_df['potencial_pct'] > 10].sort_values(
    'potencial_pct', ascending=False
)

print(f"\n💡 Oportunidades encontradas: {len(opportunities)}")

if len(opportunities) > 0:
    print(f"\nTop 5 Oportunidades de Inversión:")
    print("="*80)
    
    for idx, (i, row) in enumerate(opportunities.head(5).iterrows(), 1):
        print(f"\n🏠 Oportunidad #{idx}")
        print(f"   Precio Real: ${row['precio_real']:,.2f}")
        print(f"   Precio Predicho: ${row['precio_predicho']:,.2f}")
        print(f"   Ganancia Potencial: ${row['diferencia']:,.2f} ({row['potencial_pct']:.1f}%)")
        
        if 'bedrooms' in row:
            print(f"   Habitaciones: {int(row['bedrooms'])}")
        if 'bathrooms' in row:
            print(f"   Baños: {row['bathrooms']:.1f}")
        if 'sqft_living' in row:
            print(f"   Área: {int(row['sqft_living'])} sqft")
        if 'grade' in row:
            print(f"   Calidad: {int(row['grade'])}/13")

print("\n" + "="*80)
print("✅ ANÁLISIS COMPLETADO")


3. MODELADO Y SELECCIÓN DEL MEJOR MODELO

🤖 Modelo 1: Regresión Lineal
   - RMSE: $78,214.77
   - MAE: $52,112.92
   - R²: 0.9077

🤖 Modelo 2: Ridge Regression
   - RMSE: $78,211.88
   - MAE: $52,117.82
   - R²: 0.9077

🤖 Modelo 3: Lasso Regression


c:\Users\jonat\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.991e+12, tolerance: 1.110e+11
  model = cd_fast.enet_coordinate_descent(
c:\Users\jonat\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.936e+12, tolerance: 8.724e+10
  model = cd_fast.enet_coordinate_descent(
c:\Users\jonat\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

   - RMSE: $78,212.61
   - MAE: $52,106.10
   - R²: 0.9077

🤖 Modelo 4: Random Forest Regressor
   - RMSE: $11,782.40
   - MAE: $4,531.71
   - R²: 0.9979

🤖 Modelo 5: Gradient Boosting Regressor
   - RMSE: $11,375.50
   - MAE: $7,609.39
   - R²: 0.9980

📊 COMPARACIÓN DE MODELOS
           Modelo         RMSE          MAE       R²  CV Score (MSE)
Linear Regression 78214.767335 52112.915665 0.907708    5.846383e+09
            Ridge 78211.884274 52117.820041 0.907715    5.844676e+09
            Lasso 78212.610713 52106.102826 0.907714    5.844782e+09
    Random Forest 11782.395730  4531.709057 0.997906    2.139598e+08
Gradient Boosting 11375.501555  7609.394970 0.998048    1.448334e+08

🏆 MEJOR MODELO: Gradient Boosting
   - RMSE: $11,375.50
   - R²: 0.9980

4. IDENTIFICACIÓN DE VIVIENDAS ATRACTIVAS PARA INVERSIÓN

📈 ESTRATEGIA DE INVERSIÓN:

Una vivienda es atractiva para inversión cuando:
1. El precio de mercado está SUB-VALUADO respecto a la predicción del modelo
2. Tiene característi